# Set Ups

In [39]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
from pkg_resources import ContextualVersionConflict

# from rich.console import Console

import requests
import json

import time
from sqlalchemy import delete
from tqdm import tqdm
import re
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd

In [40]:
# connet to graphql
sample_transport=RequestsHTTPTransport(
    url='http://localhost:4001/graphql',
    use_json=True,
    headers={
        "Content-type": "application/json",
    },
    verify=False
)

client = Client(
    #execute_timeout=20,
    transport=sample_transport,
    fetch_schema_from_transport=True,
)

session = requests.Session()
session.auth = ('admin', "secret")

#hostname = 'http://ec2-3-64-237-95.eu-central-1.compute.amazonaws.com:8080/tilt/tilt'
#auth = session.post(hostname)
#response = json.loads(session.get(hostname).content)

#tilt = json.loads(requests.get('http://ec2-3-64-237-95.eu-central-1.compute.amazonaws.com:8080/tilt/tilt').content)
#print("Fetch Successful!", response)


json_file = open('/Users/johannes/Desktop/TU/playground/tilt')
tilt = json.load(json_file)

In [22]:
hostname = 'http://ec2-3-64-237-95.eu-central-1.compute.amazonaws.com:8080/tilt/tilt'
auth = session.post(hostname)
response = session.get(hostname).content
json_tilt = json.loads(response)

In [41]:
import os
import re

repo_tilts = []

directory = os.fsencode('/Users/johannes/Desktop/TU/tilt-corpus')

for folder in os.listdir(directory):
    foldername = os.fsdecode(folder)
    if len(foldername) == 1:
        folder = os.fsencode(f'/Users/johannes/Desktop/TU/tilt-corpus/{foldername}')
        for file in os.listdir(folder):
            filename = os.fsdecode(file)
            if filename.endswith("tilt.json"): 
                repo_tilts.append(json.load(open(f'/Users/johannes/Desktop/TU/tilt-corpus/{foldername}/{filename}')))
            else:
                continue
    else:
        continue

# Load Data from TILT HUB into NEO4J

In [56]:

def create_node(parent):

    """
    func:   create_node
            Recursively goes through JSON and builds a mutation to generate the neo4j graph
            Base case is when an entry is a string
    """

    def rem_dq(uncleaned):

        """
        func:   rem_dq
                removes double quotes. 
                Needed to avoid errors in upload
        """
        if isinstance(uncleaned,str):
            return re.sub('"', '', uncleaned)
        else:
            return uncleaned

    #mutation string
    label_string = ""

    #iterate through elements of dictionary or list
    for i, key in enumerate(parent):

        #if key == 'rightToRectificationOrDeletion':
            #label_string += " HERE "
            
        try:
            if key[0] == "$": # the dollar sign infront of the $oid is not recognised so we have to remove it
                label_string += f'{key[1:]}: "{rem_dq(parent[key])}"'


            # if key is a list
            elif isinstance(parent[key], list):

            
                #if list item is empty
                if len(parent[key]) == 0:
                    
                    #label_string += f'{key}: "{parent[key]}"'
                    label_string += "" # just add nothing. not ideal as we lose out on recording missing information but easier to fix than trying to fix missfilled fields in tilt

                #two types of entries: nodes or strings
                else:

                    # When we get lists with multiple same node entries, we make a list of nodes. 
                    # There we only need one create statement and then close it afterwards
                    label_string += key+ ':{create:['
                    node_list = True

                    ### !!! WE NOW HAVE A FIELD SUBCATEGORIES THAT CONSISTS OF JUST A LIST OF STRINGS. 
                    # SOMEHOW WE NEED TO JUST PARSE IT IN AS SUCH A LIST. MAYBE MERGE ALL LIST ENTRIES INTO ONE STRING. 
                    # MAYBE JUST PASS THE WHOLE LIST. NOT SURE YET. DIFFERENT FORMAT... 

                    for list_item in parent[key]:
                        
                        # some entries are strings wrapped in a list instead of a node.
                        if type(list_item) == type(""): 
                            #deleted = label_string[-len(f'{key}:{{create:['):]
                            label_string = label_string[:-len(f'{key}:{{create:[')]                             
                            label_string += f'{key}: "{rem_dq(list_item)}"'
                            node_list = False
                    

                        # if the list items are dictionaries make a new node
                        else: 
                            label_string += f'{{{create_node(list_item)}}},'

                    #
                    if node_list:
                        label_string = label_string[:-1]+']}'
            
                
            # if the element is a dictionary
            elif isinstance(parent[key], dict): 
                
                #create the next node for this dict
                label_string += f'{key}:{{create:[{{{create_node(parent[key])}}}]}}'

            else: # base case
                
                if i < len(parent)-1:

                    label_string += f'{key}: "{rem_dq(parent[key])}", '
                    
                else:
                    label_string += f'{key}: "{rem_dq(parent[key])}"'
                
        except Exception as ex:
            label_string += "BIG ERROR HERE"  
            print(ex)  

    #certain characters (like tilda) are rejected by gql
    label_string = re.sub('~', "", label_string)
    label_string = label_string.strip().replace('\n', "")
    
    return label_string

    

In [57]:
print("mutation { createtilts( input: [ { " + create_node(repo_tilts[0]) + "}] ) {tilts {meta { name } } }}")

mutation { createtilts( input: [ { meta:{create:[{_id: "1e350b3a-98e3-40cd-bfe2-f580798e341b", name: "rewe", created: "2021-06-21T12:21:52.355664", modified: "2021-06-21T12:21:52.355692", version: "1", language: "de", status: "active", url: "https://www.rewe.de/service/datenschutz/lieferservice-abholservice", _hash: "66e86086311e3b9bd0089eb4fe1aebe4fde00d6e8457e25fd251e4ea2b2a86c8"}]}controller:{create:[{name: "REWE Markt GmbH", country: "None", address: "None", division: "None", representative:{create:[{name: "None", email: "None", phone: "None"}]}sector: "G"}]}dataProtectionOfficer:{create:[{email: "datenschutz@rewe.de", phone: "None", country: "None", name: "None", address: "None"}]}dataDisclosed:{create:[{_id: "56c2dcbf-81d3-4000-98e0-ab9b7296fe25", category: "Wir verarbeiten deine personenbezogenen Daten, wenn du den REWE Liefer- oder Abholservice nutzt. Im Einzelnen könnendies folgende Daten sein:Daten über deine beim REWE Liefer- oder Abholservice oder im Markt getätigten Einkäu

In [75]:
tilt_mutations = []
for tilt_entry in tqdm(repo_tilts):
    tilt_mutations.append(("mutation { createtilts( input: [ { " + create_node(tilt_entry) + "}] ) {tilts {meta { name } } }}").replace('representative: "None",',""))
    #tilt_mutations[-1] = tilt_mutations[-1].replace('representative: "None"','')

100%|██████████| 71/71 [00:00<00:00, 1446.44it/s]


In [77]:
query = gql(tilt_mutations[4])
client.execute(query)

{'createtilts': {'tilts': [{'meta': [{'name': 'instagram'}]}]}}

In [79]:
tilt_mutations[12]

'mutation { createtilts( input: [ { meta:{create:[{_id: "2baca7b7-15da-46c2-89de-5f026a0955e7", name: "telefonica", created: "2021-06-21T12:21:52.725732", modified: "2021-06-21T12:21:52.725751", version: "1", language: "de", status: "active", url: "https://www.telefonica.de/unternehmen/datenschutz/datenschutz-auf-dem-portal.html", _hash: "9972abdc06bbfc94d92357b934302eac093eb7057a14b9a44c9b0a679a2e902f"}]}controller:{create:[{name: "Telefónica Germany GmbH & Co. OHG", country: "DE", address: "Telefónica Germany GmbH & Co. OHG  Georg-Brauchle-Ring 50  80992 München", representative:{create:[{name: "None", email: "None", phone: "None"}]}sector: "J"}]}dataProtectionOfficer:{create:[{email: "None", url: "https://www.telefonica.de/datenschutz-kontakt", country: "DE", name: "None", address: "Telefónica Germany GmbH & Co. OHG  Datenschutzbeauftragter  Georg-Brauchle-Ring 50  80992 München"}]}dataDisclosed:{create:[{_id: "31ce6f7f-8af6-4606-9482-626914db84cf", category: "Logfiles", purposes:{c

In [78]:
successes = 0
for i, query_string in enumerate(tqdm(tilt_mutations)):
    #print("Query String: ", query_string)
    try:
        query = gql(query_string)
        client.execute(query)
        successes += 1
    except Exception as syn:
        print(syn)
        print("Error With Tilt No.", i)

print("Successfully Uploaded:", 100*(successes/(i+1)), "per. of tilts")

 15%|█▌        | 11/71 [00:02<00:13,  4.30it/s]

Syntax Error: Expected ':', found Name 'account'.

GraphQL request:1:9053
1 | mutation { createtilts( input: [ { meta:{create:[{_id: "90eeb36e-3d54-4b0a-88b7-
  | f4565cd66920", name: "Zoom", created: "2021-07-17T10:17:13.671015", modified: "2
  | 021-07-17T10:17:13.671037", version: "1", language: "en", status: "active", url:
  |  "https://zoom.us/privacy", _hash: "54840059bb2a41fcca06c6381e0e6c0708ae0e0d4e97
  | 57612fe9d646820f7c1f"}]}controller:{create:[{name: "Zoom Video Communications, I
  | nc.", country: "US", address: "55 Almaden Blvd, Suite 600San Jose, CA 95113", di
  | vision: "Data", representative:{create:[{name: "None", email: "None", phone: "No
  | ne"}]}sector: "J"}]}dataProtectionOfficer:{create:[{email: "privacy@zoom.us", ph
  | one: "None", country: "US", name: "None", address: "Zoom Video Communications, I
  | nc.Attention: Data Privacy Officer55 Almaden Blvd, Suite 600San Jose, CA 95113"}
  | ]}dataDisclosed:{create:[{_id: "0112176c-e991-444e-b0ab-780f7bceaa43", c

 24%|██▍       | 17/71 [00:02<00:06,  8.17it/s]

Syntax Error: Expected ':', found Name 'usubcategories'.

GraphQL request:1:3493
1 | mutation { createtilts( input: [ { meta:{create:[{_id: "129ada7f-4fe4-4088-ad7c-
  | 43aecde1764e", name: "Tagesschau App", created: "2021-07-30T11:43:04.642821", mo
  | dified: "2021-07-30T11:43:04.642850", version: "1", language: "de", status: "act
  | ive", url: "https://www.tagesschau.de/datenschutzerklaerung100.html", _hash: "3a
  | 3d07cd967cf1e6c7a2fabd4e8ab087230c7fd3747fe457f01717b864c2e0be"}]}controller:{cr
  | eate:[{name: "Norddeutscher Rundfunk (NDR)", country: "Norddeutsch", address: "R
  | othenbaumchaussee 13220149 Hamburg", division: "Anstalt des öffentlichen Rechts"
  | , representative:{create:[{name: "Der Intendant", email: "https://www.tagesschau
  | .de/mehr/kontakt/kontakt-uebersicht-101.html"}]}sector: "J"}]}dataProtectionOffi
  | cer:{create:[{email: "datenschutz@ndr.de", country: "Norddeutsch", name: "Dr. He
  | iko Neuhoff", address: "GremienbüroRothenbaumchaussee 13220149 Ha

 41%|████      | 29/71 [00:03<00:02, 14.47it/s]

...

Syntax Error: Unexpected ':'.

GraphQL request:1:3961
1 | mutation { createtilts( input: [ { meta:{create:[{_id: "811eeae0-fd1b-469d-ae4e-
  | 4c970988aee1", name: "Bundestag.de", created: "2021-07-21T13:45:48.230697", modi
  | fied: "2021-07-21T13:45:48.230720", version: "1", language: "de", status: "activ
  | e", url: "https://www.bundestag.de/datenschutz", _hash: "4f6e67c7814699269550361
  | 95b9b4db91ccd23ccc516f08cf180ff2ada205b3b"}]}controller:{create:[{name: "Deutsch
  | er Bundestag", country: "DE", address: "Deutscher BundestagPlatz der Republik 1D
  |  - 11011 Berlin", division: "None", representative:{create:[{name: "None", email
  | : "mail@bundestag.de", phone: "+49 30 227-0"}]}sector: "O"}]}dataProtectionOffic
  | er:{create:[{email: "datenschutz.bdb@bundestag.de", phone: "+49 30 227-0", count
  | ry: "DE", name: "Verantwortlicher für die Datenverarbeitung", address: "Deutsche
  | r BundestagPlatz der Republik 1"}]}dataDisclosed:{create:[{_id: "60a89516-46ea-4
  | 8bf-8df

 87%|████████▋ | 62/71 [00:11<00:00,  9.97it/s]

Syntax Error: Expected ':', found Name 'Kontaktdaten'.

GraphQL request:1:3762
1 | mutation { createtilts( input: [ { meta:{create:[{_id: "45fa45a5-39b6-4149-b953-
  | 8600b96b4cc0", name: "Luca App", created: "2021-07-30T11:41:37.194017", modified
  | : "2021-07-30T11:41:37.194046", version: "1", language: "de", status: "active", 
  | url: "https://www.luca-app.de/app-privacy-policy", _hash: "2e9253ce62c796f3e5bda
  | d5605d3b9212003affa0bf097f305c933afa3d8bac9"}]}controller:{create:[{name: "cultu
  | re4life GmbH", country: "Deutschland", address: "culture4life GmbHMörikestraße 6
  | 770199 StuttgartDeutschland", division: "None", representative:{create:[{name: "
  | None", email: "info@culture4life.de", phone: "None"}]}sector: "J"}]}dataProtecti
  | onOfficer:{create:[{email: "privacy@culture4life.de", phone: "None", country: "D
  | eutschland", name: "culture4life GmbH", address: "Charlottenstraße 5910117 Berli
  | nDeutschland"}]}dataDisclosed:{create:[{_id: "fe3abf2d-3b87-48f3-81

100%|██████████| 71/71 [01:07<00:00,  1.05it/s]

Syntax Error: Expected Name, found '{'.

GraphQL request:1:4611
1 | mutation { createtilts( input: [ { meta:{create:[{_id: "b2326270-15ad-4a3c-afbb-
  | 317b43c752c8", name: "webde", created: "2021-06-21T12:21:52.509774", modified: "
  | 2021-06-21T12:21:52.509794", version: "1", language: "de", status: "active", url
  | : "https://agb-server.web.de/datenschutz", _hash: "db271845c6bdb0fe0d37195b1cafd
  | 2ec72566b87534e8f187099a2951a7fce27"}]}controller:{create:[{name: "1&1 Mail & Me
  | dia GmbH", country: "DE", address: "1&1 Mail & Media GmbH, Zweigniederlassung Ka
  | rlsruhe, Brauerstr. 48, 76135 Karlsruhe, Deutschland", division: "Zweigniederlas
  | sung Karlsruhe", representative:{create:[{name: "Alexander Charles, Ralf Harting
  | s, Thomas Ludwig, Jan Oetjen", email: "info@web.de", phone: "(+49) 07219609740"}
  | ]}sector: "J"}]}dataProtectionOfficer:{create:[{email: "datenschutz@webde.de", p
  | hone: "None", country: "DE", name: "Datenschutzbeauftragter der 1&1 Mail & Media
 

# Analysis 

In [9]:
from neo4j import GraphDatabase

In [11]:
class make_queries:

    """
    class:  exampleTilt
    input:  - uri: bolt uri from neo4j
            - auth: credentials for neo4j
            - no_nodes: number of nodes to create 
            - comp_df:  dataframe with company names and websites
            - mu_poisson_*: average no. of samples for poisson distribution
            - reset: whether to delete all nodes in database (default False, meant for experimentation)

    """

    def __init__(self, uri, auth,  reset = False):
        self.driver = GraphDatabase.driver(uri, auth=auth)
        self.reset = reset


    def close(self):
        self.driver.close()
        
    # Match and display all friendships.
    @classmethod
    def print_friendships(cls, tx):
        result = tx.run("MATCH (a)-[:KNOWS]->(b) RETURN a.name, b.name")
        for record in result:
            print("{} knows {}".format(record["a.name"], record["b.name"]))

    @classmethod
    def delete_nodes(cls, tx):
        tx.run("MATCH (n) detach delete n")

    
    def get_sorensen(self, tx, threshold = .9):
        tx.run("MATCH (m:meta), (r:recipient) "
                f"WHERE apoc.text.sorensenDiceSimilarity(apoc.text.clean(m.name), apoc.text.clean(r.name), 'en') > {threshold} "
                "MERGE (m)-[c:has_sorensen]->(r)")


    def run_query(self, tx, cyper_statement):
        
        result = tx.run(cyper_statement)
        return [point.data() for point in result] 
        

    def main(self):
        saved_bookmarks = []  # To collect the session bookmarks

        if self.reset:
            assurance = input("WARNING DELETING ALL DATA FROM THE GRAPH, TO CONTINUE WRITE 'yes'")
            if assurance == 'yes':
                with self.driver.session() as session_del:
                    session_del.write_transaction(self.delete_nodes)
                    saved_bookmarks.append(session_del.last_bookmark())

        #with self.driver.session() as session_sorensen: 
        #    session_sorensen.write_transaction(self.get_sorensen)

infos = make_queries(uri = 'bolt://localhost:7687', 
                    auth = ('neo4j', 'letmein'), 
                    reset=False)

In [12]:
dict_of_categories = [{'category':i['d.category'],'name':i['m.name']} for i in infos.driver.session().write_transaction(infos.run_query, "match (m:meta)-[]-(t:tilt)-[]->(d:dataDisclosed) return m.name, d.category")]# if i['d.category'] != None ]

list_of_categories = [i['category'] for i in dict_of_categories  if i['category'] != None ]

with open('categories_tilt', 'wb') as fp:
    pickle.dump(np.unique(np.asarray(list_of_categories)), fp)

In [13]:
plt.rcParams.update(plt.rcParamsDefault)

plt.rc('font',**{'family':'serif','serif':['Times']})
plt.rc('text', usetex=True)

In [15]:
grouped_by_name_df = pd.DataFrame(dict_of_categories).groupby('name').count()
plt.grid()
plt.hist(grouped_by_name_df['category'], color = 'black')
plt.style.use('seaborn-pastel')

plt.title(f"Number of categories per controller \n Mean: {round(grouped_by_name_df['category'].sum()/73)}")
plt.ylabel("Frequency")
plt.xlabel("No. of categories")
plt.savefig("No_categories per controller")

In [16]:
dict_of_purposes = [{'purpose':i['p.purpose'],'name': i['m.name']} for i in infos.driver.session().write_transaction(infos.run_query, "match (m:meta)-[]-(t:tilt)-[]->(d:dataDisclosed)-[]-(p:purposes) return p.purpose, m.name")]# if i['p.purpose'] != None ]

list_of_purposes =  [i['purpose'] for i in dict_of_purposes if i['purpose'] != None]

with open('purpose_tilt', 'wb') as fp:
    pickle.dump(np.unique(np.asarray(list_of_purposes)), fp)

In [17]:
pd_purposes = pd.DataFrame(dict_of_purposes).groupby('name').count()
plt.grid()
plt.hist(pd_purposes['purpose'], color= 'black')
plt.title(f"No. of purposes per controller \n Mean (including firms with None): {round(pd_purposes['purpose'].sum()/73)}")
plt.ylabel('Frequency')
plt.xlabel("No. purposes")
plt.savefig("no_purposes_per_controller")

In [18]:
pd_purposes.head()

,purpose
name,
ARD Mediathek,6
Amazon Alexa Terms of Use,4
Apple,2
Bundestag.de,14
CDU,0


In [19]:
pd.DataFrame(dict_of_purposes)

,purpose,name
0,None,fresenius medical care
1,None,fresenius medical care
2,None,fresenius medical care
3,None,fresenius medical care
4,None,fresenius medical care
...,...,...
1337,None,enel
1338,None,enel
1339,None,enel
1340,None,enel
